In [10]:
%run imports_and_functions.py
from collections import defaultdict, Counter
check_overlap=False

In [11]:
folder_path = '/media/eoserver/AnaO_ATL/JP_ATL/mhw_labeled_dataset/2d_detection/' #params.getp('output_folder')  # replace with your folder path
substring = '.'
files_with_substring = get_files_with_substring(folder_path, substring)
files = [folder_path + file for file in files_with_substring]

NEW TRY WHERE YOU DONT EDIT ANYTHING!! JUST RECORD NEW START AND END AND IN THE NEXT PHASE THE PROBLEM IS FIXED!!!

In [12]:
dataset = xr.open_mfdataset(files, chunks={'time': 200})
time_ds = dataset.time.values

In [ ]:
cut_dates = dict() #record start, end, and cut stamps of mhws

prev_frame = dataset.sel(time=time_ds[0])['mhw_label'].values
prev_unique = list(np.unique(prev_frame))
if check_overlap: prev_unique.remove(0)

for lbl in prev_unique:
    if lbl not in cut_dates.keys():
        cut_dates[lbl] = list()
        cut_dates[lbl].append(time_ds[0])

for date in time_ds[1:]:
    date = to_datetime(date); print(date.strftime('%Y-%m-%d'))

    frame = dataset.sel(time=date)['mhw_label'].values

    frame_lbls = list(np.unique(frame))

    if check_overlap:
        aux_labels=frame_lbls.copy(); aux_labels.remove(0)
        #exception for last date: close every date
        if date==to_datetime(time_ds[-1]):   
            for aux_lbl in aux_labels:
                if aux_lbl in cut_dates.keys():
                    cut_dates[aux_lbl].append(date)
        
        # get new start_dates
        for lbl in aux_labels:
            if lbl not in cut_dates.keys():
                cut_dates[lbl] = list()
                cut_dates[lbl].append(date)

        #encode combination of all possible labels
        unique_codes, _ = get_unique_code(prev_unique, frame_lbls)
        #get the pairing count for each label and total_overlap
        label_match, total_overlap, _, _, _ = get_pairing_count(prev_frame, frame, prev_unique, frame_lbls, unique_codes)

        # get new end dates ########################################################
        for lbl, overlap_count in label_match.items():
            # overlap condition OR mhw ended in prev_frame
            if total_overlap[lbl]==0 or overlap_count/total_overlap[lbl] < 0.5:   #or overlap_count< min_pixels_per_frame
                cut_dates[lbl].append(date-timedelta(days=1))
    else:
        frame_lbls.remove(0)
        
        #open labels
        for lbl in frame_lbls:
            if lbl not in cut_dates.keys():
                cut_dates[lbl] = list()
                cut_dates[lbl].append(date)
        
        #close labels   
        for lbl in prev_unique:
            if lbl not in frame_lbls:
                cut_dates[lbl].append(date-timedelta(days=1))
                
        if date == time_ds[-1]:
            for lbl in frame_lbls:
                cut_dates[lbl].append(date)
            

    ###################################################################################### 
    prev_frame = frame
    prev_unique = frame_lbls
        
dataset.close()

if 0 in cut_dates.keys(): del cut_dates[0] #just in case

if check_overlap: lbl_dates_dir = '/home/eoserver/beatriz/JP/data/lbl_dates.pkl'
else: lbl_dates_dir = '/home/eoserver/beatriz/JP/data/lbl_dates_no_overlap.pkl'

with open(lbl_dates_dir, 'wb') as outf:
    pickle.dump(cut_dates, outf)

In [ ]:
for lbl, date_list in cut_dates.items():
    if len(date_list)<2:
        print(f'{lbl}: {date_list}')